In [1]:
#Import packages
import geemap
import geopandas
import ee
import os

In [2]:
Map = geemap.Map(center=[20,85], zoom=4)
Map

Map(center=[20, 85], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Tog…

In [3]:
#Load in aoi
bounding_box = 'C:/Users/Jeremy/OneDrive/GIS/Blender/Ananya Map/Output/bounding_box.shp'
aoi = geemap.shp_to_ee(bounding_box)
aoi_geometry = aoi.geometry()

#Add aoi_geometry bounding box to the map
Map.addLayer(aoi_geometry, {}, 'aoi')

#Centre the map over the aoi
Map.centerObject(aoi_geometry, 11) 
  
#Dates over which to create a median composite. Full dates are biased towards summer due to cloud coverage.
start_date = ee.Date('2017-01-01')
end_date = ee.Date('2023-05-30')

#Set cloud filtering variables
CLOUD_FILTER = 60
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50

In [4]:
def get_s2_sr_cld_col(aoi_geometry, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi_geometry)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi_geometry)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

In [5]:
s2_sr_cld_col = get_s2_sr_cld_col(aoi_geometry, start_date, end_date)

In [6]:
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

In [7]:
def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')))

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

In [8]:
def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

In [9]:
def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

In [10]:
s2_sr_median = (s2_sr_cld_col.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .median())

In [11]:
#Visualise the composite image
Map.addLayer(s2_sr_median, {'bands':['B4',  'B3',  'B2'], 'min':0, 'max':3000, 'gama':1.1}, 'Sentinel 2 Composite')

In [12]:
#Sample region to test fishnet download
region = ee.Geometry.BBox(85.5439, 25.0891, 87.0342, 27.6858)
Map.addLayer(region, {}, 'ROI')
Map.centerObject(region)

In [13]:
#Create a fishnet for parallel downloading
fishnet = geemap.fishnet(region, h_interval=0.1, v_interval=0.1, delta=0.1)

#Style and add the fishnet to the map
style = {'color': 'ffff00ff', 'fillColor': '00000000'}
Map.addLayer(fishnet.style(**style), {}, 'Fishnet')

In [15]:
#Parallel download the S2
#output="C:/Users/Jeremy/OneDrive/GIS/Blender/Ananya Map/Assets/S2/S2_SR.tif"
geemap.download_ee_image_tiles(
    s2_sr_median, 
    fishnet, 
    out_dir="C:/Users/Jeremy/OneDrive/GIS/Blender/Ananya Map/Assets/S2", 
    scale=1000, 
    crs='EPSG:3857'
)

001.tif: |          | 0.00/15.0k (raw) [  0.0%] in 00:00 (eta:     ?)

There is no STAC entry for: None


002.tif: |          | 0.00/15.0k (raw) [  0.0%] in 00:00 (eta:     ?)

003.tif: |          | 0.00/16.2k (raw) [  0.0%] in 00:00 (eta:     ?)